In [ ]:
import concurrent.futures
import warnings
from datetime import datetime
from functools import reduce
from pathlib import Path

import iris
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.optimize import minimize

from wildfires.analysis import *
from wildfires.data import *
from wildfires.logging_config import enable_logging
from wildfires.qstat import *
from wildfires.utils import *

if "TQDMAUTO" in os.environ:
    from tqdm.auto import tqdm
else:
    from tqdm import tqdm

enable_logging("jupyter")
figure_saver = FigureSaver(directories=Path("~") / "tmp" / "interp_comp", debug=True)

warnings.filterwarnings("ignore", ".*Collapsing a non-contiguous coordinate.*")
warnings.filterwarnings("ignore", ".*DEFAULT_SPHERICAL_EARTH_RADIUS.*")
warnings.filterwarnings("ignore", ".*guessing contiguous bounds.*")
warnings.filterwarnings("ignore", ".*divide by zero.*")

mpl.rc("figure", figsize=(12, 6))

## Create datasets

In [ ]:
variables = ("SWI(1)", "FAPAR", "LAI", "VOD Ku-band", "SIF")
datasets = Datasets(
    (Copernicus_SWI(), MOD15A2H_LAI_fPAR(), VODCA(), GlobFluo_SIF())
).select_variables(variables)

### Temporal interpolation

In [ ]:
timeperiod = (datetime(2008, 1, 1, 0, 0), datetime(2015, 4, 1, 0, 0))
period_str = f"{timeperiod[0]:%Y-%m} - {timeperiod[1]:%Y-%m}"
for dataset in datasets:
    dataset.limit_months(*timeperiod)

In [ ]:
datasets.show("pretty")

In [ ]:
assert all(cube.shape[0] == datasets.cubes[0].shape[0] for cube in datasets.cubes[1:])

### Combined mask

In [ ]:
total_masks = []

for var in tqdm(variables, desc="Variable"):
    cube = datasets.select_variables(var, inplace=False).cube.copy()
    if not cube.coords("month_number"):
        iris.coord_categorisation.add_month_number(cube, "time")

    # Ignore areas that are always masked, e.g. water.
    ignore_mask = np.all(cube.data.mask, axis=0)

    # Also ignore those areas with low data availability.
    ignore_mask |= np.sum(cube.data.mask, axis=0) > (
        7 * 6
        + 10  # Up to 6 months for each of the 7 complete years.  # Extra Jan, Feb, Mar, Apr, X, X, X, X, X, X
    )

    total_masks.append(ignore_mask)

combined_mask = reduce(
    np.logical_or, [regrid(dummy_lat_lon_cube(mask)).data for mask in total_masks]
)

In [ ]:
# Plot combined ignore masks.
fig = cube_plotting(
    combined_mask,
    title=f"Combined Mask\n{period_str}",
    colorbar_kwargs={"label": "masked"},
    boundaries=np.linspace(0, 1, 3),
    fig=plt.figure(figsize=(18, 9)),
)
figure_saver.save_figure(fig, f"combined mask samples")

In [ ]:
ba_dataset = GFEDv4()
ba_dataset.limit_months(*timeperiod)
mean_ba = ba_dataset.cube.collapsed("time", iris.analysis.MEAN)
mean_ba.data.mask = ~get_land_mask()

In [ ]:
fig = cube_plotting(
    mean_ba,
    title=f"Mean BA\n{period_str}",
    colorbar_kwargs={"label": "BA", "format": "%0.0e"},
    cmap="YlOrRd",
    fig=plt.figure(figsize=(18, 9)),
    boundaries=[1e-5, 1e-4, 1e-3, 1e-2, 1e-1],
    extend="min",
)
figure_saver.save_figure(fig, f"burned area")

In [ ]:
masked_mean_ba = mean_ba.copy()
masked_mean_ba.data.mask = combined_mask
fig = cube_plotting(
    masked_mean_ba,
    title=f"Mean BA\n{period_str}",
    colorbar_kwargs={"label": "BA", "format": "%0.0e"},
    cmap="YlOrRd",
    fig=plt.figure(figsize=(18, 9)),
    boundaries=[1e-5, 1e-4, 1e-3, 1e-2, 1e-1],
    extend="min",
)
figure_saver.save_figure(fig, f"combined mask burned area")

### Apply combined mask to 'fresh' datasets and get data filled using minima and season-trend model

In [ ]:
masked_datasets = Datasets(
    (Copernicus_SWI(), MOD15A2H_LAI_fPAR(), VODCA(), GlobFluo_SIF())
).select_variables(variables)

# Select correct time period and regrid to common grid.
for dataset in masked_datasets:
    dataset.limit_months(*timeperiod)
    dataset.regrid()

# Apply the combined mask.
masked_datasets.apply_masks(combined_mask)

# Retrieve the filled dataset for later comparison.
processed_datasets = Datasets(
    [dataset.get_persistent_season_trend_dataset() for dataset in masked_datasets]
)

In [ ]:
orig_datasets = Datasets(
    (Copernicus_SWI(), MOD15A2H_LAI_fPAR(), VODCA(), GlobFluo_SIF())
).select_variables(variables)
for dataset in orig_datasets:
    dataset.limit_months(*timeperiod)
    dataset.regrid()

In [ ]:
for var in orig_datasets.pretty_variable_names:
    cube_plotting(
        orig_datasets.select_variables(var, inplace=False).cube.collapsed(
            "time", iris.analysis.MEAN
        )
        - processed_datasets.select_variables(
            f"{var} 50P 4k", inplace=False
        ).cube.collapsed("time", iris.analysis.MEAN),
        title=f"Orig - Filled {var}",
    )

In [ ]:
for var in orig_datasets.pretty_variable_names:
    orig_cube = orig_datasets.select_variables(var, inplace=False).cube
    filled_cube = processed_datasets.select_variables(
        f"{var} 50P 4k", inplace=False
    ).cube
    mean_diff = orig_cube.collapsed("time", iris.analysis.MEAN) - filled_cube.collapsed(
        "time", iris.analysis.MEAN
    )
    for i in (np.abs(mean_diff.data.ravel())).argsort(fill_value=-1)[::-1][:10]:
        s = (slice(None), *np.unravel_index(i, mean_diff.shape))
        plt.figure()
        plt.title(var)
        plt.plot(filled_cube.data[s], label="filled")
        plt.plot(orig_cube.data[s], label="orig")
        plt.legend(loc="best")